<hr>

# PREDICTING THE STOCK MARKET WITH WATSON

## Part 2: Introduction

In this Jupyter Notebook you'll learn step-by-step how to use the Watson Machine Learning API that was automatically generated when the previously created WS Modeler flow was deployed. You will also learn how to download files from IBM Cloud Object Storage and generate interactive visualizations using `Bokeh`. 

This Notebook is part of the series on stock market forecasting.

## Table of Contents

#### 1. Using the Watson Machine Learning Python Client
* 1.1: Setting up WML Credentials and Client
* 1.2: Preparing Input Data
* 1.3: Making an API Call to WML
* 1.4: Parsing the WML results

#### 2. Visualizing the Results
* 2.1: Plotting the Modeler Flow Forecasts
* 2.2: Validating Modeler Flow Forecasts with Observed Data
* 2.3: Interacting with Complete Historic and Forecasted Data

<hr>

# 1: Using the Watson Machine Learning Python Client

Previously, we trained a time series forecaster in Watson Modeler Flow and later deployed this forecaster in a Watson Machine Learning service instance.

Now, in this section, we will use the WML API to send new input data to our time series forecaster.

### 1.1: Setting up WML Credentials and Client

* Once you are on IBM Cloud Pak for Data, on the top right corner click on your avatar, and then click on `Profile and settings`. Go to the `Services` tab.

If the `Machine Learning` service instance is not added then click `Add` and note down the name of the `Machine Learning` service instance associated with your `Cloud Pak for Data`. You will need to provide this name in future steps.


In [ ]:
!pip uninstall watson-machine-learning-client -y | tail -n 1
!pip install --user watson-machine-learning-client-v4==1.0.115 --upgrade | tail -n 1
!pip install --user pyspark==2.3.3 --upgrade | tail -n 1

### 4.1 Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**<font color='red'><< Enter your `api_key` and `location` in the following cell. >></font>>**

In [ ]:
api_key = '<api_key>'
location = '<location>'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

Now, paste the scoring_endpoint link you copied in the first part of this code pattern into the variable below.

In [ ]:
scoring_endpoint = "<scoring endpoint from deployment>"

In [ ]:
# Instaling the Watson Machine Learning Python Client
# !pip install --user watson-machine-learning-client==1.0.376 --upgrade

In [ ]:
# Create a WML client
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(wml_credentials)

In [ ]:
wml_client.spaces.list()

### 4.2 Use the desired space as the `default_space`

The deployment space ID will be looked up based on the name specified above. If you do not receive a space GUID as an output to the next cell, do not proceed until you have created a deployment space.

**<font color='red'><< UPDATE THE VARIABLE 'MODEL_NAME' TO A UNIQUE NAME>></font>**

**<font color='red'><< UPDATE THE VARIABLE 'DEPLOYMENT_SPACE_NAME' TO THE NAME OF THE DEPLOYMENT SPACE CREATED PREVIOUSLY>></font>**

In [ ]:
MODEL_NAME = "aapl_forecast"
DEPLOYMENT_SPACE_NAME = "Sanjeev WML Deployment space"

In [ ]:
wml_client.spaces.list()
all_spaces = wml_client.spaces.get_details()['resources']
space_id = None
for space in all_spaces:
    if space['entity']['name'] == DEPLOYMENT_SPACE_NAME:
        space_id = space["metadata"]["id"]
        print("\nDeployment Space ID: ", space_id)

if space_id is None:
    print("WARNING: Your space does not exist. Create a deployment space before proceeding to the next cell.")
    #space_id = client.spaces.store(meta_props={client.spaces.ConfigurationMetaNames.NAME: space_name})["metadata"]["guid"]

In [ ]:
wml_client.set.default_space(space_id)

In [ ]:
wml_client.deployments.list()

In [ ]:
deployment_id='b74da7ec-2430-46f9-9d8b-aea7b91186ab'

### 1.2: Preparing Input Data

The `payload` dict will contain only some points of data for demonstration. 

This payload must be a dict type with the same structure as the csv file prepared with Data Refinery in Watson Studio.

Remember that the `WIKI/TABLE` Quandl database only goes until 27-March-2018.

In [ ]:
# NOTE: manually define and pass the array(s) of values to be scored in the next line
payload = {wml_client.deployments.ScoringMetaNames.INPUT_DATA:
    [{ "fields": ["Date", "Open", "High", "Low", "Close"],
    "values": [['2017-03-28', 140.91, 144.04, 140.62, 143.80]]
     }]
}

### 1.3: Making an API Call to WML

The next code cell executes a HTTP request to WML with the payload as input.

In [ ]:
data = wml_client.deployments.score(deployment_id, payload)

In [ ]:
print(data)

Before using `Bokeh` to interact with the data, we need to parse it in a Pandas dataframe.

### 1.4: Loading data into pandas

In [ ]:
columns = data["predictions"][0]["fields"]
print(columns)

In [ ]:
values = data["predictions"][0]["values"]
print(values)

Next we just create a new Pandas dataframe with the future data for Apple Inc. stocks, retrieved from WML using the API.

In [ ]:
import pandas as pd

ndf = pd.DataFrame.from_records(values, columns=columns)
print(ndf.info())
ndf.tail()

<hr>

# 2: Validating and Visualizing the Results

In [ ]:
!pip install --user bokeh==1.0.4 --upgrade

In [ ]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.embed import components
from bokeh.io import output_notebook

print('Packages imported.')

In [ ]:
# Load bokeh
output_notebook()

### 2.1: Plotting the Modeler Flow Forecasts

In [ ]:
# Figure
p = figure(plot_width=1200, plot_height=550, title='Historic and Predicted Stock Value Data', x_axis_type="datetime")

# Plot Lines
p.line(ndf.Date, ndf['$TS-Close'], line_width=3, line_color="#ff6699", legend='Modeled Close Value')
p.line(ndf.Date, ndf['$TS-Open'], line_width=3, line_color="#0099ff", legend='Modeled Open Value')
p.line(ndf.Date, ndf['$TSLCI-Close'], line_width=0.5, line_color="#ff6699", legend='Modeled Close Value Bounds')
p.line(ndf.Date, ndf['$TSUCI-Close'], line_width=0.5, line_color="#ff6699", legend='Modeled Close Value Bounds')
p.line(ndf.Date, ndf['$TSLCI-Open'], line_width=0.5, line_color="#0099ff", legend='Modeled Open Value Bounds')
p.line(ndf.Date, ndf['$TSUCI-Open'], line_width=0.5, line_color="#0099ff", legend='Modeled Open Value Bounds')

# Axis and Labels
p.legend.orientation = "vertical"
p.legend.location = "top_left"
p.xaxis.axis_label = "Date"
p.xaxis.axis_label_text_font_style = 'bold'
p.xaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.axis_label = "Value ($ USD)"
p.yaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '12pt'

In [ ]:
show(p)

### 2.3: Validating Modeler Flow Forecasts with Observed Data

Click on the `0100` button on the top right corner here in Watson Studio, and select the `AAPL.csv_shaped.csv` file and then Insert to code -> Insert pandas DataFrame.

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# TODO
# Click on the 0100 button on the top right corner here in Watson Studio, and select the AAPL.csv_shaped.csv file and then Insert to code -> Insert pandas DataFrame.




In [ ]:
# Now we split the full historical data into a train and a test dataset:
import datetime

split_date = datetime.date(2015, 1, 2)
#df_train = df_data_1[(pd.to_datetime(df_data_1["Date"]) < split_date)]
df_test = df_data_1[(pd.to_datetime(df_data_1["Date"]) > split_date)]
df_test['Date'] = df_test['Date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'))
df_test = df_test.sort_values(['Date'])
df_test.tail()

In [ ]:
# Figure
p = figure(plot_width=1200, plot_height=550, title='Historic and Predicted Stock Value Data', x_axis_type="datetime")

# Plot Lines
p.line(ndf.Date, ndf['$TS-Close'], line_width=3, line_color="#ff6699", legend='Forecasted Close Value')
p.line(ndf.Date, ndf['$TS-Open'], line_width=3, line_color="#0099ff", legend='Forecasted Open Value')
p.line(df_test.Date, df_test.Close, line_width=0.5, line_color="#ff6699", legend='Historic Close Data (Test Sample)')
p.line(df_test.Date, df_test.Open, line_width=0.5, line_color="#0099ff", legend='Historic Open Data (Test Sample)')

# Axis and Labels
p.legend.orientation = "vertical"
p.legend.location = "top_left"
p.xaxis.axis_label = "Date"
p.xaxis.axis_label_text_font_style = 'bold'
p.xaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.axis_label = "Value ($ USD)"
p.yaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '12pt'

In [ ]:
show(p)

In [ ]:
ndf_filtered = ndf.drop(['Close', 'Open', '$TSResidual-Open', '$TSResidual-Close'], axis=1)

result = pd.concat([ndf_filtered, df_test], axis=1).dropna()
result = result.loc[:,~result.columns.duplicated()]
result = result.sort_values(['Date'])
result.tail()

In the next cell, simple mean errors are calculated (percentual and absolute):

In [ ]:
open_abs_errors = []
close_abs_errors = []
open_pct_errors = []
close_pct_errors = []

for index, row in result.iterrows():
    open_abs_errors.append(abs(row['Open']-row['$TS-Open']))
    close_abs_errors.append(abs(row['Close']-row['$TS-Close']))
    open_pct_errors.append((abs(row['Open']-row['$TS-Open']))/row['Open'])
    close_pct_errors.append((abs(row['Close']-row['$TS-Close']))/row['Close'])
    
mean_open_error = sum(open_abs_errors) / len(open_abs_errors)
mean_close_error = sum(close_abs_errors) / len(close_abs_errors)
mean_open_pct_error = sum(open_pct_errors) / len(open_pct_errors)
mean_close_pct_error = sum(close_pct_errors) / len(close_pct_errors)

print('Mean Errors in 1-Year Future Prediction:')
print('Analyzed Stock: AAPL (Apple Inc.)')
print('----------------------------------------')
print('Mean Open Value Error (USD): {} $'.format(round(mean_open_error, 3)))
print('Mean Close Value Error (USD): {} $'.format(round(mean_close_error, 3)))
print('Mean Open Value Error: {}%'.format(round(mean_open_pct_error*100, 3)))
print('Mean Close Value Error: {}%'.format(round(mean_close_pct_error*100, 3)))

### 2.3: Interacting with Complete Historic and Forecasted Data

In [ ]:
# Figure
p = figure(plot_width=1200, plot_height=550, title='Historic and Predicted Stock Value Data', x_axis_type="datetime")

# Plot Lines
p.line(ndf.Date, ndf['$TSLCI-Close'], line_width=0.5, line_color="#ff6699", legend='Modeled Close Value Bounds')
p.line(ndf.Date, ndf['$TSUCI-Close'], line_width=0.5, line_color="#ff6699", legend='Modeled Close Value Bounds')
p.line(ndf.Date, ndf['$TSLCI-Open'], line_width=0.5, line_color="#0099ff", legend='Modeled Open Value Bounds')
p.line(ndf.Date, ndf['$TSUCI-Open'], line_width=0.5, line_color="#0099ff", legend='Modeled Open Value Bounds')

p.line(ndf.Date, ndf['$TS-Close'], line_width=3, line_color="#ff6699", legend='Forecasted Close Value')
p.line(ndf.Date, ndf['$TS-Open'], line_width=3, line_color="#0099ff", legend='Forecasted Open Value')
p.line(df_test['Date'], df_test['Close'], line_width=0.5, line_color="#ff6699", legend='Historic Close Data (Test Sample)')
p.line(df_test['Date'], df_test['Open'], line_width=0.5, line_color="#0099ff", legend='Historic Open Data (Test Sample)')

# Axis and Labels
p.legend.orientation = "vertical"
p.legend.location = "top_left"
p.xaxis.axis_label = "Date"
p.xaxis.axis_label_text_font_style = 'bold'
p.xaxis.axis_label_text_font_size = '16pt'
p.xaxis.major_label_text_font_size = '14pt'
p.yaxis.axis_label = "Value ($ USD)"
p.yaxis.axis_label_text_font_style = 'bold'
p.yaxis.axis_label_text_font_size = '16pt'
p.yaxis.major_label_text_font_size = '12pt'

In [ ]:
show(p)

<hr>

This notebook and its source code is made available under the terms of the <a href = "https://github.com/IBM/watson-stock-market-predictor/blob/master/LICENSE">Apache License 2.0</a>.

<hr>

### Thank you for completing this journey!